In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.4/640.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.6/228.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 65.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): 

In [2]:
%%bash
pip install farm-haystack[ocr]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!apt-get install poppler-utils 
!pip install pdf2image

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (225 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 128285 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [5]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [6]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

Need to let the server startup for 30s

In [7]:
import time
time.sleep(30)

In [8]:
from haystack.utils import launch_es
launch_es()

In [9]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable  HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [10]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

indexing_pipeline = Pipeline()
# text_converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
pdf_converter = PDFToTextConverter(remove_numeric_tables=True)
# doc_converter = DocxToTextConverter(remove_numeric_tables=False, valid_languages=["en"])
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
import os

# indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=pdf_converter, name="PDFToTextConverter", inputs=["File"])
# indexing_pipeline.add_node(component=doc_converter, name="DocConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["PDFToTextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

In [12]:
iss_dir = "data/iss"

files_to_index = [iss_dir + "/" + f for f in os.listdir(iss_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/2 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/2 [00:00<?, ?docs/s]

{'documents': [<Document: {'content': '\nMTech EBAC, IS and SE\nNUS-ISS\nRULES, REGULATIONS &\nPROCEDURES\n© 2023 NUS.\nThe contents contained in this\ndocument may not be reproduced in\nany form or by any means, without\nthe written permission of NUS-ISS\nother than the purpose for which it\nhas been supplied.\n25 Heng Mui Keng Terrace,\x0c\nTable of Contents\n1.2. Organisation of the Document.............................................................................................4\n2. EXAMINATION AND CAPSTONE PROJECT REGULATIONS ....................................................5\n2.3. Organisation and Conduct of Examinations......................................................................5\n2.4. Illness Prior or During an Examination...............................................................................7\n2.5. Absence from an Examination.............................................................................................8\n2.6. Failing an Examination and/o

In [13]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

In [14]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [15]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [16]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])


In [34]:
prediction = querying_pipeline.run(
    query="What happens if I fall sick?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium` and `all`
)

from pprint import pprint

pprint(prediction)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]


Query: What happens if I fall sick?
Answers:
[   {   'answer': 'the student will be permitted to\n'
                  'leave the examination to seek medical attention',
        'context': 'ls ill during an examination, then the student will be '
                   'permitted to\n'
                   'leave the examination to seek medical attention. The '
                   'student should report to a\n'
                   're'},
    {   'answer': 'completes the examination',
        'context': 'of a student who is ill during an examination but '
                   'nevertheless\n'
                   'completes the examination, this fact may be taken into '
                   'account by the Board of\n'
                   'Examiner'},
    {   'answer': 'dismissed from the programme',
        'context': 'ficates will not be permitted to\n'
                   'proceed further and will be dismissed from the '
                   'programme.\x0c'
                   '\n'
                   '

In [20]:
%%bash

nvidia-smi

Thu Mar 23 16:48:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |   1799MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
# from pprint import pprint

# pprint(prediction)

In [19]:
# from haystack.utils import print_answers

# print_answers(
#     prediction,
#     details="minimum" ## Choose from `minimum`, `medium` and `all`
# )